# EXAMPLE DATA

In [11]:
# number of risky assets
num_assets = 3

# means
mn1 = 0.08
mn2 = 0.12
mn3 = 0.15

# std devs
sd1 = 0.15
sd2 = 0.25
sd3 = 0.35

# correlations
c12 = 0.15
c13 = 0.60
c23 = 0.30

# risk-free rate
rf = 0.02

# or savings rate and borrowing rate
rs = 0.02
rb = 0.05

# risk aversion
raver = 6

# INSTALL LIBRARIES

In [ ]:
!pip install cvxopt

# CREATE ARRAYS

In [12]:
import numpy as np 

mn = np.array([mn1, mn2, mn3,])
sd = np.array([sd1, sd2, sd3])

corr = np.identity(3)
corr[0, 1] = corr[1, 0] = c12
corr[0, 2] = corr[2, 0] = c13
corr[1, 2] = corr[2, 1] = c23

cov = np.diag(sd) @ corr @ np.diag(sd)

# CHECK CORRELATIONS

In [13]:
if np.all(np.linalg.eigvals(cov) > 0):
    print("correlations are acceptable")
else:
    print("correlations are inconsistent")

correlations are acceptable


# OPTIMAL PORTFOLIO 1

Use this if the borrowing and savings rate are the same and short sales are allowed.

In [21]:
optimum = np.linalg.solve(raver*cov, mn-rf)
optimum_mn = rf + optimum @ (mn-rf)
optimum_sd = np.sqrt(optimum @ cov @ optimum)

print(f"optimal savings is {max(0, 1-np.sum(optimum)): 0.3f}")
print(f"optimal borrowing is {max(0, np.sum(optimum)-1): 0.3f}")
print(f"optimal portfolio of risky assets is {optimum}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal standard deviation is {optimum_sd: 0.3f}")

optimal savings is  0.411
optimal borrowing is  0.000
optimal portfolio of risky assets is [0.32455745 0.21782746 0.04673581]
optimal expected return is  0.067
optimal std dev is  0.089


# OPTIMAL PORTFOLIO 2

Use this if the borrowing rate is greater than the savings rate and short sales are allowed.    

In [27]:
from cvxopt import matrix
from cvxopt.solvers import qp

# to define variance penalty
P = np.zeros((num_assets+2, num_assets+2))
P[2:,2:] = raver*cov

# to define (minus) expected return 
q = np.concatenate(
    (
        np.array([-rs, rb]),
        -mn
    )
)

# to impose -xs <= 0 and -xb <= 0
G = np.zeros((2, num_assets+2))
G[0, 0] = G[1, 1] = -1
h = np.zeros(2)

# to impose xs - xb + w1 + w2 + w3 = 1
A = np.ones(num_assets+2)
A[1] = -1
b = [1.]

# create cvxopt matrix objects
P = matrix(P, (num_assets+2, num_assets+2))
q = matrix(q, (num_assets+2, 1))
G = matrix(G, (2, num_assets+2))
h = matrix(h, (2, 1))
A = matrix(A, (1, num_assets+2))
b = matrix(b, (1, 1))

# compute optimum by quadratic programming
solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
optimum = np.array(solution["x"])
xs, xb = optimum[0].item(), optimum[1].item()
optimum = optimum[2:].reshape(-1, )

# expected return and risk
optimum_mn = xs*rs - xb*rb + optimum @ mn
optimum_sd = np.sqrt(optimum @ cov @ optimum)

print(f"optimal savings is {xs: 0.3f}")
print(f"optimal borrowing is {xb: 0.3f}")
print(f"optimal portfolio of risky assets is {optimum}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal standard deviation is {optimum_sd: 0.3f}")

     pcost       dcost       gap    pres   dres
 0: -4.0439e-02 -4.5721e-03  2e+00  2e+00  1e+00
 1: -1.5522e-02 -9.4543e-02  8e-02  4e-02  4e-02
 2: -3.2415e-02 -4.4584e-02  1e-02  1e-15  2e-17
 3: -4.3552e-02 -4.3904e-02  4e-04  4e-16  6e-18
 4: -4.3665e-02 -4.3668e-02  4e-06  2e-16  1e-17
 5: -4.3666e-02 -4.3666e-02  4e-08  2e-16  2e-17
Optimal solution found.
optimal savings is  0.411
optimal borrowing is  0.000
optimal portfolio of risky assets is [0.32455691 0.21782731 0.04673589]
optimal expected return is  0.067
optimal standard deviation is  0.089


# OPTIMAL PORTFOLIO 3

Use this if short sales are not allowed.  If the borrowing and savings rate are the same (=rf) then add the line:

    rb = rs = rf

In [30]:
from cvxopt import matrix
from cvxopt.solvers import qp

# to define variance penalty
P = np.zeros((num_assets+2, num_assets+2))
P[2:,2:] = raver*cov

# to define (minus) expected return 
q = np.concatenate(
    (
        np.array([-rs, rb]),
        -mn
    )
)

# to impose -xs <= 0 and -xb <= 0 and -w <= 0
G = -np.identity(num_assets+2)
h = np.zeros(num_assets+2)

# to impose xs - xb + w1 + w2 + w3 = 1
A = np.ones(num_assets+2)
A[1] = -1
b = [1.]

# create cvxopt matrix objects
P = matrix(P, (num_assets+2, num_assets+2))
q = matrix(q, (num_assets+2, 1))
G = matrix(G, (num_assets+2, num_assets+2))
h = matrix(h, (num_assets+2, 1))
A = matrix(A, (1, num_assets+2))
b = matrix(b, (1, 1))

# compute optimum by quadratic programming
solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
optimum = np.array(solution["x"])
xs, xb = optimum[0].item(), optimum[1].item()
optimum = optimum[2:].reshape(-1, )

# expected return and risk
optimum_mn = xs*rs - xb*rb + optimum @ mn
optimum_sd = np.sqrt(optimum @ cov @ optimum)

print(f"optimal savings is {xs: 0.3f}")
print(f"optimal borrowing is {xb: 0.3f}")
print(f"optimal portfolio of risky assets is {optimum}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal std dev is {optimum_sd: 0.3f}")

     pcost       dcost       gap    pres   dres
 0: -4.8185e-02 -4.8058e-01  7e+00  3e+00  3e+00
 1:  1.0156e-01 -8.5350e-01  1e+00  3e-01  3e-01
 2:  5.2138e-03 -1.3843e-01  1e-01  3e-15  1e-16
 3: -4.0064e-02 -8.0442e-02  4e-02  9e-16  5e-17
 4: -3.4882e-02 -5.5854e-02  2e-02  2e-16  2e-17
 5: -4.3490e-02 -4.4356e-02  9e-04  2e-16  1e-17
 6: -4.3663e-02 -4.3712e-02  5e-05  2e-16  2e-17
 7: -4.3666e-02 -4.3666e-02  6e-07  6e-17  1e-17
 8: -4.3666e-02 -4.3666e-02  6e-09  1e-16  2e-17
Optimal solution found.
optimal savings is  0.411
optimal borrowing is  0.000
optimal portfolio of risky assets is [0.3245571  0.21782738 0.04673607]
optimal expected return is  0.067
optimal std dev is  0.089
